<a href="https://colab.research.google.com/github/nairsatish/4540/blob/master/Copy_of_demistifying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From article: https://www.analyticsvidhya.com/blog/2020/02/mathematics-behind-convolutional-neural-network/

In [1]:
#importing required libraries
import numpy as np
import pandas as pd
import tqdm as tqdm
from keras.datasets import mnist

#loading dataset
(x_train, y_train), (x_test,y_test) = mnist.load_data()

#selecting a subset of data (200 images)
x_train = x_train[:200]
y=y_train[:200]

X = x_train.T
X = X/255

y.resize((200,1))
y = y.T
#checking value
pd.Series(y[0]).value_counts()


11490434/11490434 [==============================] - 0s 0us/step


1    26
9    23
0    21
4    21
3    21
7    21
2    20
6    19
8    15
5    13
dtype: int64

In [2]:
#converting into binary classification
for i in range(y.shape[1]):
    if y[0][i] >4:
        y[0][i] = 1
    else:
        y[0][i] = 0

#checking value counts
pd.Series(y[0]).value_counts()

0    109
1     91
dtype: int64

In [3]:
# initializing filter
f=np.random.uniform(size=(3,5,5))
f = f.T

print('Filter 1', '\n', f[:,:,0], '\n')
print('Filter 2', '\n', f[:,:,1], '\n')
print('Filter 3', '\n', f[:,:,2], '\n')

Filter 1 
 [[0.76633236 0.1225288  0.36818648 0.42750291 0.66442129]
 [0.40534081 0.44788935 0.38034627 0.86028342 0.39126656]
 [0.87394139 0.96888082 0.82937973 0.16568721 0.23903279]
 [0.02019277 0.04649089 0.12813062 0.4066673  0.43510968]
 [0.65747512 0.22178052 0.66985237 0.67201996 0.10943952]] 

Filter 2 
 [[0.73403472 0.58492787 0.1696076  0.9145459  0.10011524]
 [0.35742651 0.26346369 0.0870383  0.51100503 0.27146994]
 [0.80818169 0.47476575 0.35181852 0.48014837 0.99611978]
 [0.7628365  0.24956039 0.74918333 0.04944729 0.31362556]
 [0.66936497 0.09795262 0.64957318 0.95045454 0.66101262]] 

Filter 3 
 [[0.79041304 0.06028521 0.21101204 0.90864704 0.32880175]
 [0.20864798 0.47698257 0.93674651 0.50868743 0.79594557]
 [0.15798715 0.96349607 0.30374291 0.37724413 0.12643985]
 [0.59018332 0.40705621 0.80601098 0.86749828 0.85826459]
 [0.74046046 0.30067374 0.52402519 0.60405781 0.44822833]] 



In [4]:
X.shape, y.shape, f.shape

((28, 28, 200), (1, 200), (5, 5, 3))

In [5]:
# Generating patches from images
new_image = []

# for number of images
for k in range(X.shape[2]):
    # sliding in horizontal direction
    for i in range(X.shape[0]-f.shape[0]+1):
        # sliding in vertical direction
        for j in range(X.shape[1]-f.shape[1]+1):
            new_image.append(X[:,:,k][i:i+f.shape[0],j:j+f.shape[1]])
            
# resizing the generated patches as per number of images
new_image = np.array(new_image)
new_image.resize((X.shape[2],int(new_image.shape[0]/X.shape[2]),new_image.shape[1],new_image.shape[2]))
new_image.shape

(200, 576, 5, 5)

In [6]:
# number of features in data set
s_row = X.shape[0] - f.shape[0] + 1
s_col = X.shape[1] - f.shape[1] + 1
num_filter = f.shape[2]

inputlayer_neurons = (s_row)*(s_col)*(num_filter)
output_neurons = 1 

# initializing weight
wo=np.random.uniform(size=(inputlayer_neurons,output_neurons))

In [7]:
# defining the Sigmoid Function
def sigmoid (x):
    return 1/(1 + np.exp(-x))

# derivative of Sigmoid Function
def derivatives_sigmoid(x):
    return x * (1 - x)

In [8]:
# generating output of convolution layer
filter_output = []
# for each image
for i in range(len(new_image)):
    # apply each filter
    for k in range(f.shape[2]):
        # do element wise multiplication
        for j in range(new_image.shape[1]):
            filter_output.append((new_image[i][j]*f[:,:,k]).sum()) 

filter_output = np.resize(np.array(filter_output), (len(new_image),f.shape[2],new_image.shape[1]))

# applying activation over convolution output
filter_output_sigmoid = sigmoid(filter_output)

filter_output.shape, filter_output_sigmoid.shape

((200, 3, 576), (200, 3, 576))

In [9]:
# generating input for fully connected layer
filter_output_sigmoid = filter_output_sigmoid.reshape((filter_output_sigmoid.shape[0],filter_output_sigmoid.shape[1]*filter_output_sigmoid.shape[2]))

filter_output_sigmoid = filter_output_sigmoid.T

# Linear trasnformation for fully Connected Layer
output_layer_input= np.dot(wo.T,filter_output_sigmoid)
output_layer_input = (output_layer_input - np.average(output_layer_input))/np.std(output_layer_input)

# activation function
output = sigmoid(output_layer_input)

In [10]:
#Error
error = np.square(y-output)/2

#Error w.r.t Output (Gradient)
error_wrt_output = -(y-output)

#Error w.r.t sigmoid transformation (output_layer_input)
output_wrt_output_layer_input=output*(1-output)

#Error w.r.t weight
output_wrt_w=filter_output_sigmoid

In [11]:
#delta change in w for fully connected layer
delta_error_fcp = np.dot(output_wrt_w,(error_wrt_output * output_wrt_output_layer_input).T)
lr = 0.01
wo = wo - lr*delta_error_fcp

In [12]:
#Error w.r.t sigmoid output
output_layer_input_wrt_filter_output_sigmoid = wo.T

#Error w.r.t sigmoid transformation
filter_output_sigmoid_wrt_filter_output = filter_output_sigmoid * (1-filter_output_sigmoid)

# cvalculating derivatives for backprop convolution
error_wrt_filter_output = np.dot(output_layer_input_wrt_filter_output_sigmoid.T,error_wrt_output*output_wrt_output_layer_input) * filter_output_sigmoid_wrt_filter_output
error_wrt_filter_output = np.average(error_wrt_filter_output, axis=1)
error_wrt_filter_output = np.resize(error_wrt_filter_output,(X.shape[0]-f.shape[0]+1,X.shape[1]-f.shape[1]+1, f.shape[2]))

In [13]:
filter_update = []
for i in range(f.shape[2]):
    for j in range(f.shape[0]):
        for k in range(f.shape[1]):            
            temp = 0
            spos_row = j
            spos_col = k
            epos_row = spos_row + s_row
            epos_col = spos_col + s_col
            for l in range(X.shape[2]):
                temp = temp + (X[spos_row:epos_row,spos_col:epos_col,l]*error_wrt_filter_output[:,:,i]).sum()
            filter_update.append(temp/X.shape[2])  

filter_update_array = np.array(filter_update)
filter_update_array = np.resize(filter_update_array,(f.shape[2],f.shape[0],f.shape[1]))

In [14]:
for i in range(f.shape[2]):
    f[:,:,i] = f[:,:,i] - lr*filter_update_array[i]

In [15]:
lst = ['string', 1, 2, 'another string']
for element in lst:
   print type(element)

SyntaxError: ignored